# 自然言語処理入門

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_5/intro-to-nlp.ipynb)

テキストデータの処理と分析の基礎である自然言語処理を概観しましょう。

まず、テキストデータの前処理を段階を踏んで見ていきます。

次に、簡単な例から自然言語処理の考え方を学びます。

## テキストデータの前処理

テキストデータを分析する際には、基本的に以下の処理が行われます。

1. 分かち書き（形態素解析）
2. 品詞付与
3. 係り受け解析
4. 固有表現抽出
5. 原形抽出


形態素は、「言葉が意味を持つまとまりの単語の最小単位」で、形態素解析は、文章を一つ一つの形態素に分ける技術です。単語が区切られていない日本語などの言語では特に重要です。

日本語を例に、`spaCy`と`GiNZA`というライブラリを使って処理の過程を見ていきましょう。

`spaCy`では上の一連の処理をまとめて行ってくれます。

テキストデータとして、オー・ヘンリー（結城浩訳）[『最後の一枚の葉』](https://www.hyuki.com/trans/leaf.html)の冒頭部分を使います。

In [ ]:
text = """
ワシントン・スクエア西にある小地区は、 道路が狂ったように入り組んでおり、 「プレース」と呼ばれる区域に小さく分かれておりました。 この「プレース」は不可思議な角度と曲線を描いており、 一、二回自分自身と交差している通りがあるほどでした。 かつて、ある画家は、この通りが貴重な可能性を持っていることを発見しました。 例えば絵や紙やキャンバスの請求書を手にした取り立て屋を考えてみてください。 取り立て屋は、この道を歩き回ったあげく、 ぐるりと元のところまで戻ってくるに違いありません。 一セントも取り立てることができずにね。
それで、芸術家たちはまもなく、奇妙で古いグリニッチ・ヴィレッジへとやってきました。 そして、北向きの窓と十八世紀の切り妻とオランダ風の屋根裏部屋と安い賃貸料を探してうろついたのです。 やがて、彼らは しろめ製のマグやこんろ付き卓上なべを一、二個、六番街から持ち込み、 「コロニー」を形成することになりました。
ずんぐりした三階建ての煉瓦造りの最上階では、スーとジョンジーがアトリエを持っていました。 「ジョンジー」はジョアンナの愛称です。 スーはメイン州の、ジョンジーはカリフォルニア州の出身でした。 二人は八番街の「デルモニコの店」の定食で出会い、 芸術と、チコリーのサラダと、ビショップ・スリーブの趣味がぴったりだとわかって、 共同のアトリエを持つことになったのでした。
"""

In [ ]:
# ライブラリのインストール

!pip install spacy ginza ja-ginza
!pip install sklearn pandas

In [ ]:
# ライブラリのインポート
import spacy

# 日本語モデルのロード
nlp = spacy.load("ja_ginza")

# 解析
doc = nlp(text)

# 結果の確認
for token in doc:
    print(token)

形態素解析の結果には、語の原形や品詞の情報も含まれます。

In [ ]:
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

係り受け（単語の修飾関係）は、次のように確認できます。

In [ ]:
import pandas as pd

# 解析結果をpandasのDataFrameに入れる

df = pd.DataFrame({
    "text": token.text,
    "lemma_": token.lemma_,
    "pos_": token.pos_,
    "tag_": token.tag_,
    "dep_": token.dep_,
    "children": list(token.children)
} for token in doc)

In [ ]:
df

In [ ]:
# 係り受けの図を表示する

spacy.displacy.render(doc, style="dep")

それでは、単語の使用頻度を数えてみましょう。

In [ ]:
from collections import Counter

# 単語の頻度を数える
counter = Counter(token.lemma_ for token in doc)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

句読点や助詞など、意味がなさそうな言葉ばかりです。

より意味がある語を取り出すために、分析対象とする品詞を指定しましょう。具体的には、内容語である名詞、動詞、形容詞（、固有名詞）を指定すればよいでしょう。

In [ ]:
# 分析対象とする品詞の指定
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc if token.pos_ in include_pos)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ちょっとよくなりました。でも、「こと」「ある」「いる」などの一般的な名詞や動詞が多いように思えます。

これらを不要語として指定し、除去しましょう。

In [ ]:
# 分析対象とする品詞と不要語（ストップワード）を指定する
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")
stopwords = ("する", "ある", "おる", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc
                  if token.pos_ in include_pos and token.lemma_ not in stopwords)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ずっと良くなりました。これだけの作業で、Bag-of-Wordsを作成することができました。

---

## 自然言語処理とは

日本語や英語など、私たちが普段使っている言葉を自然言語（Natural Language）と言います。自然言語処理（Natural Language Processing）とは、自然言語を処理する分野です。

自然言語処理の目標は、人の話す言葉をコンピュータに理解させ、私たちにとって役に立つことをコンピュータに行わせることです。

私たちの言葉は「文字」によって表現することができます。そして、言葉の意味は「単語」（正確には形態素）によって構成されます。そのため、自然言語をコンピュータに理解させるためには、「単語の意味」を理解させることが重要です。

ここでは、統計情報から単語の意味を表現する手法を学びます。

## 統計的な手法

自然言語処理の研究や応用のために目的をもって収集されたテキストデータを「コーパス」と呼びます。WikipediaやGoogle Newsなどのテキストデータや、シェイクスピアや夏目漱石などの作品群もコーパスです。

コーパスはテキストデータであり、そこに含まれる文章は人によって書かれたものです。これはつまり、コーパスには自然言語に対する人の知識が含まれているということです。文章の書き方、単語の選び方、単語の意味などがコーパスには含まれています。

カウントベースの手法の目標は、人の知識が詰まったコーパスから、自動的に、効率よく、そのエッセンスを抽出することです。

## 簡単なコーパスの前処理

テキストデータを単語に分割し、分割した単語を単語IDのリストへ変換することで、データの前処理を行いましょう。

In [ ]:
text = "You say goodbye and I say hello."

In [ ]:
# 小文字に変換
text = text.lower()
text

In [ ]:
# 句点の前にスペースを挿入
text = text.replace(".", " .")
text

In [ ]:
# 文を単語に分割する
words = text.split(' ')
words

In [ ]:
# 単語のIDと単語の対応表を作る

word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [ ]:
word_to_id

In [ ]:
id_to_word

この2つの辞書を使えば、単語から単語IDの検索と、単語IDから単語の検索ができます。

In [ ]:
id_to_word[2]

In [ ]:
word_to_id["i"]

最後に、単語のリストを単語IDのリストに変換し、NumPy配列に変換します。

In [ ]:
import numpy as np

corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

これでコーパスの前処理は終了です。

## 単語の分散表現


次に、コーパスを使って単語の意味を抽出しましょう。具体的には、単語をベクトルで表すことを目指します。これは、自然言語処理の分野では、単語の分散表現と呼ばれます。

単語の分散表現に関する手法は、「単語の意味は、周囲の単語によって形成される」というアイデアに基づいています。これは、分布仮説と呼ばれるものです。

分布仮説では、単語自体には意味がなく、その単語の「コンテキスト（文脈）」によって、単語の意味が形成されると言われています。

たしかに、意味的に同じ単語は、同じような文脈で多く出現します。

例えば、

* I drink beer.
* We drink wine.

のようにdrinkの近くには飲み物があらわれやすいでしょう。


* I guzzle beer.
* We guzzle wine.

のような文章では、guzzleという単語がdrinkと同じような文脈で使われていることが分かります。そして、guzzleとdrinkが近い意味の単語だということが導けます。

## 共起行列

分布仮説に基づいて、単語をベクトルで表す方法を考えます。

素直な方法は、周囲の単語を数えることです。

さきほど用意したコーパスに含まれるそれぞれの単語について、そのコンテキスト（目当ての単語の周囲）に含まれる単語の頻度を数えていきます。

例えば、youという単語に着目すると、

||you|say|goodbye|and|i|hello|.|
|---|---|---|---|---|---|---|---|
|you|0|1|0|0|0|0|0|

このような表現になります。

全ての語に対してこれを数えると、

||you|say|goodbye|and|i|hello|.|
|---|---|---|---|---|---|---|---|
|you|0|1|0|0|0|0|0|
|say|1|0|1|0|1|1|0|
|goodbye|0|1|0|1|0|0|0|
|and|0|0|1|0|1|0|0|
|i|0|1|0|1|0|0|0|
|hello|0|1|0|0|0|0|1|
|.|0|0|0|0|0|1|0|

となります。

これをNumPy配列にすることで、共起行列ができます。この共起行列を使うと、各単語の分散表現が求められます。

In [ ]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
])

In [ ]:
# youの分散表現
print(id_to_word[0], C[0])

In [ ]:
# andの分散表現
print(id_to_word[3], C[3])

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
        
    return co_matrix

単語の分散表現を使うことで、単語の類似度を計算するなど、より高度な処理ができるようになります。

## tf-idf

テキストデータの可視化には主に単語の出現頻度を使いました。これは、多く出現する単語ほど重要である、という直観的な考えに基づく指標です。

しかし、一方で、いろいろな文書に多く出現すると予測される「わたし」などの言葉は、出現頻度のわりにさほど重要ではないと考えられます。このような言葉を不要語として指定し、分析から除外することもできますが、この点を考慮した単語の重要度の指標として、tf-idfがあります。

tf-idfは、term frequency（単語頻度）-inverse document frequency（逆文書頻度）の略です。具体的には、「ある文書内である単語がどれくらい多い頻度で出現するか」を表すterm frequencyと、「全文書内である単語を含む文書がどれくらい少ない頻度で出現するか」を表すinverse document frequencyをかけ合わせた値です。

簡単に言うと、どの文書にもよく出てくる単語の重要度を下げて、あまり出てこない単語の重要度を上げるための工夫です。

tf-idfはヒューリスティックな指標で、理論的な根拠はあまりありませんが、様々な場面でうまく重要語を抽出でき、文書を特徴づけることができることが経験的に知られています。

tf-idfについてもっと詳しく見る：

[https://atmarkit.itmedia.co.jp/ait/articles/2112/23/news028.html](https://atmarkit.itmedia.co.jp/ait/articles/2112/23/news028.html)